## Problem 3: How long distance individuals have travelled? (8 points)

In this problem the aim is to calculate the "distance" in meters that the individuals have travelled according the social media posts (Euclidean distances between points). In this problem, we will need the `userid` -column and the points created in the previous problem. You will need the shapefile `Kruger_posts.shp` generated in Problem 2 as input file.

Our goal is to answer these questions based on the input data:
- What was the shortest distance travelled in meters?
- What was the mean distance travelled in meters?
- What was the maximum distance travelled in meters?

**In your code, you should first:**
 - Import required modules.
 - Read in the shapefile as a geodataframe called `data`
 - Reproject the data from WGS84 projection into `EPSG:32735` -projection which stands for UTM Zone 35S (UTM zone for South Africa) to transform the data into the metric system.
 
*Store the result in a variable called `data`*!

In [19]:
import os
import geopandas as gpd

In [20]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
def load_data(path):
    assert os.path.isfile(path), 'Input must a GIS data file'
    return gpd.read_file(path)

data = load_data('data/Kruger_posts.shp')

- Check the crs of the input data. If this information is missing, set it as epsg:4326 (WGS84).
- Reproject the data from WGS84 to `EPSG:32735` -projection which stands for UTM Zone 35S (UTM zone for South Africa) to transform the data into metric system. (don't create a new variable, update the existing variable `data`!)"

In [21]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
data.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [22]:
data.to_crs(epsg=32735, inplace=True)
data.crs

<Projected CRS: EPSG:32735>
Name: WGS 84 / UTM zone 35S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 24°E to 30°E - by country
- bounds: (24.0, -80.0, 30.0, 0.0)
Coordinate Operation:
- name: UTM zone 35S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [23]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check the data
print(data.head())

         lat        lon         timestamp    userid  \
0 -24.980792  31.484633  2015-07-07 03:02  66487960   
1 -25.499225  31.508906  2015-07-07 03:18  65281761   
2 -24.342578  30.930866  2015-03-07 03:38  90916112   
3 -24.854614  31.519718  2015-10-07 05:04  37959089   
4 -24.921069  31.520836  2015-10-07 05:19  27793716   

                            geometry  
0  POINT (-4695752.719 14973674.275)  
1  POINT (-4748939.258 15014098.837)  
2  POINT (-4672729.591 14859391.193)  
3  POINT (-4679391.656 14969037.444)  
4  POINT (-4686373.982 14973910.589)  


In [24]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check that the crs is correct after re-projecting (should be epsg:32735)
print(data.crs)

epsg:32735


 - Group the data by userid

In [25]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
grouped = data.groupby('userid')

In [26]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the number of groups:
assert len(grouped.groups) == data["userid"].nunique(), "Number of groups should match number of unique users!"

**Create LineString objects for each user connecting the points from oldest to latest:**

*Suggested steps:*
- Create an empty DataFrame called `movements`. 
- Create an empty column "geometry"
- Use a for-loop where you iterate over the grouped object. For each user's data: 
    - [sort](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html) the rows by timestamp 
    - create a LineString object based on the user's points
    - Add the LineString to the geometry column of the `movements` dataframe. You can also add the `userid` in a separate column (or use the userid as index).
- Convert `movements` into a `GeoDataFrame` (you can replace the DataFrame created in the previous steps with the GeoDataFrame). Remember to set the `geometry` column.
- Set the CRS of the ``movements`` GeoDataFrame as ``EPSG:32735`` 

In [41]:
import pandas as pd
from shapely.geometry import LineString

In [61]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
movements = pd.DataFrame()
#sort = data.sort_values(by=['timestamp'])
movements['geometry'] = None
movements['userid'] = None
coords = []
for key, grp in grouped:
    tsort = grp.sort_values(by=['timestamp'])
    #ins = {'userid': key, 'geometry': LineString([[x, y] for x, y in zip(tsort['lat'], tsort['lon'])])}
    coords.append({'userid': key, 'points':[(x, y) for x, y in zip(tsort['lat'], tsort['lon'])]})
    
print(coords)

[{'userid': 16301, 'points': [(-24.759508333, 31.3712), (-24.749845, 31.338316667), (-24.995803333, 31.592), (-24.791483333, 31.865171667), (-24.76017, 31.33943), (-25.102336167, 31.894695), (-24.774158333, 31.380341667), (-24.985141667, 31.625661667), (-25.122811167, 31.911866667)]}, {'userid': 26589, 'points': [(-25.493720243, 30.985913773)]}, {'userid': 29322, 'points': [(-25.493266016, 31.726971969)]}, {'userid': 42181, 'points': [(-25.30595914, 31.080977814)]}, {'userid': 45136, 'points': [(-25.321310937, 31.025820481), (-25.321310937, 31.025820481)]}, {'userid': 48971, 'points': [(-25.47135, 30.95611)]}, {'userid': 50136, 'points': [(-24.769847681, 31.394466839), (-24.992527641, 31.592609421), (-24.992527641, 31.592609421), (-24.769847681, 31.394466839), (-24.769847681, 31.394466839), (-24.992527641, 31.592609421), (-24.992527641, 31.592609421), (-24.769847681, 31.394466839), (-24.992527641, 31.592609421), (-24.992527641, 31.592609421)]}, {'userid': 50530, 'points': [(-24.2758864

In [29]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the result
print(type(movements))
print(movements.crs)
print(movements["geometry"].head())

<class 'pandas.core.frame.DataFrame'>


AttributeError: 'DataFrame' object has no attribute 'crs'

**Finally:**
- Check once more the crs definition of your dataframe (should be epsg:32735, define the correct crs if this information is missing)
- Calculate the lenghts of the lines into a new column called ``distance`` in ``movements`` GeoDataFrame.

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

In [ ]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the output
movements.head()

You should now be able to print answers to the following questions: 

 - What was the shortest distance travelled in meters?
 - What was the mean distance travelled in meters?
 - What was the maximum distance travelled in meters?

- Finally, save the movements of into a Shapefile called ``some_movements.shp``

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
raise NotImplementedError()

In [ ]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

import os

#Check if output file exists
assert os.path.isfile(fp), "Output file does not exits."

That's all for this week!